<a href="https://colab.research.google.com/github/Nishviprp/citibike-star-schema/blob/main/notebooks/transform_star_schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip -q install holidays pyarrow

import os, glob, zipfile
import pandas as pd
import numpy as np
import holidays

In [15]:
BASE_DIR = "/content"
RAW_DIR = os.path.join(BASE_DIR, "raw")
OUT_DIR = os.path.join(BASE_DIR, "outputs")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# Move uploaded zip/csv/parquet into RAW_DIR
for fp in glob.glob("/content/*"):
    if os.path.isfile(fp) and fp.lower().endswith((".zip", ".csv", ".parquet")):
        os.rename(fp, os.path.join(RAW_DIR, os.path.basename(fp)))

print("RAW_DIR files:")
for f in sorted(glob.glob(os.path.join(RAW_DIR, "*"))):
    print(" -", os.path.basename(f))

RAW_DIR files:
 - JC-202506-citibike-tripdata.csv.zip
 - JC-202507-citibike-tripdata.csv.zip
 - JC-202508-citibike-tripdata.csv.zip
 - JC-202509-citibike-tripdata.csv.zip
 - JC-202510-citibike-tripdata.zip
 - JC-202511-citibike-tripdata.csv.zip


In [16]:
for z in glob.glob(os.path.join(RAW_DIR, "*.zip")):
    with zipfile.ZipFile(z, "r") as zip_ref:
        zip_ref.extractall(RAW_DIR)

print("After unzip, RAW_DIR contains:")
for f in sorted(glob.glob(os.path.join(RAW_DIR, "*"))):
    print(" -", os.path.basename(f))

After unzip, RAW_DIR contains:
 - JC-202506-citibike-tripdata.csv
 - JC-202506-citibike-tripdata.csv.zip
 - JC-202507-citibike-tripdata.csv
 - JC-202507-citibike-tripdata.csv.zip
 - JC-202508-citibike-tripdata.csv
 - JC-202508-citibike-tripdata.csv.zip
 - JC-202509-citibike-tripdata.csv
 - JC-202509-citibike-tripdata.csv.zip
 - JC-202510-citibike-tripdata.csv
 - JC-202510-citibike-tripdata.zip
 - JC-202511-citibike-tripdata.csv
 - JC-202511-citibike-tripdata.csv.zip
 - __MACOSX


In [17]:
csv_files = sorted([f for f in glob.glob(os.path.join(RAW_DIR, "*.csv")) if "__MACOSX" not in f])

if not csv_files:
    raise ValueError("No CSV files found after unzip. Check your uploads.")

dfs = []
for fp in csv_files:
    df = pd.read_csv(fp)
    df["source_file"] = os.path.basename(fp)
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)
print("Loaded shape:", df_raw.shape)
print("Columns:", df_raw.columns.tolist())
df_raw.head(3)

Loaded shape: (610291, 14)
Columns: ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', 'source_file']


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
0,A21A97FF7957594F,classic_bike,2025-06-03 19:48:09.552,2025-06-03 20:07:44.535,Montgomery St,JC099,Newport PATH,JC066,40.719420,-74.050990,40.727224,-74.033759,casual,JC-202506-citibike-tripdata.csv
1,59CDAC3D90F9187B,electric_bike,2025-06-08 20:34:05.505,2025-06-08 20:44:13.290,Dixon Mills,JC076,Bergen Ave,JC095,40.721630,-74.049968,40.722104,-74.071455,casual,JC-202506-citibike-tripdata.csv
2,E7655D4BABFD1245,classic_bike,2025-06-25 22:00:10.302,2025-06-25 22:06:27.373,Bergen Ave & Sip Ave,JC109,Bergen Ave,JC095,40.731009,-74.064437,40.722104,-74.071455,member,JC-202506-citibike-tripdata.csv


In [18]:
df = df_raw.copy()

# Parse datetime
df["started_at"] = pd.to_datetime(df["started_at"], errors="coerce")
df["ended_at"]   = pd.to_datetime(df["ended_at"], errors="coerce")
df = df.dropna(subset=["started_at", "ended_at"]).copy()

# Duration
df["duration_min"] = (df["ended_at"] - df["started_at"]).dt.total_seconds() / 60.0
df = df[df["duration_min"] > 0].copy()

# ride_id must be integer (your JC file has hex strings)
df["ride_id"] = pd.factorize(df["ride_id"].astype(str))[0] + 1
df["ride_id"] = df["ride_id"].astype(int)

# Keep ONLY station names in fact (drop ids + lat/long, etc.)
needed = [
    "ride_id","rideable_type","member_casual",
    "started_at","ended_at",
    "start_station_name","end_station_name",
    "duration_min","source_file"
]
df = df[needed].copy()

# Clean station names
df["start_station_name"] = df["start_station_name"].astype(str).str.strip()
df["end_station_name"]   = df["end_station_name"].astype(str).str.strip()

print("Cleaned shape:", df.shape)
df.head(3)

Cleaned shape: (610284, 9)


,ride_id,rideable_type,member_casual,started_at,ended_at,start_station_name,end_station_name,duration_min,source_file
0,1,classic_bike,casual,2025-06-03 19:48:09.552,2025-06-03 20:07:44.535,Montgomery St,Newport PATH,19.583050,JC-202506-citibike-tripdata.csv
1,2,electric_bike,casual,2025-06-08 20:34:05.505,2025-06-08 20:44:13.290,Dixon Mills,Bergen Ave,10.129750,JC-202506-citibike-tripdata.csv
2,3,classic_bike,member,2025-06-25 22:00:10.302,2025-06-25 22:06:27.373,Bergen Ave & Sip Ave,Bergen Ave,6.284517,JC-202506-citibike-tripdata.csv


In [19]:
def time_of_day_label(hour: int) -> str:
    # EDIT THIS if your professor defines different bins
    if 5 <= hour <= 11:
        return "Morning"
    elif 12 <= hour <= 16:
        return "Afternoon"
    elif 17 <= hour <= 20:
        return "Evening"
    else:
        return "Night"

df["start_hour"] = df["started_at"].dt.hour
df["time_of_day"] = df["start_hour"].apply(time_of_day_label)

df[["started_at","start_hour","time_of_day"]].head(5)

,started_at,start_hour,time_of_day
0,2025-06-03 19:48:09.552,19,Evening
1,2025-06-08 20:34:05.505,20,Evening
2,2025-06-25 22:00:10.302,22,Night
3,2025-06-05 05:04:38.203,5,Morning
4,2025-06-19 00:16:23.490,0,Night


In [20]:
# If you later switch to NYC data, keep NY holidays:
# For JC you could use NJ, but assignment wants NYC -> NY is appropriate.
state_code = "NY"
us_holidays = holidays.US(subdiv=state_code)

df["date"] = df["started_at"].dt.date
df["date_key"] = df["started_at"].dt.strftime("%Y%m%d").astype(int)

dim_date = (
    df[["date", "date_key"]]
    .drop_duplicates()
    .sort_values("date")
    .reset_index(drop=True)
)

dt_series = pd.to_datetime(dim_date["date"])
dim_date["day_num"] = dt_series.dt.day
dim_date["day_name"] = dt_series.dt.day_name()
dim_date["month_num"] = dt_series.dt.month
dim_date["month_name"] = dt_series.dt.month_name()
dim_date["year"] = dt_series.dt.year
dim_date["quarter_num"] = dt_series.dt.quarter

def holiday_name(d):
    return us_holidays.get(d, "")

dim_date["holiday_name"] = dim_date["date"].apply(holiday_name)

dim_date.head(5)

,date,date_key,day_num,day_name,month_num,month_name,year,quarter_num,holiday_name
0,2025-05-31,20250531,31,Saturday,5,May,2025,2,
1,2025-06-01,20250601,1,Sunday,6,June,2025,2,
2,2025-06-02,20250602,2,Monday,6,June,2025,2,
3,2025-06-03,20250603,3,Tuesday,6,June,2025,2,
4,2025-06-04,20250604,4,Wednesday,6,June,2025,2,


In [21]:
def make_dim(df_in, col, dim_name):
    values = (
        df_in[[col]].dropna()
        .drop_duplicates()
        .sort_values(col)
        .reset_index(drop=True)
    )
    values[f"{dim_name}_key"] = np.arange(1, len(values) + 1, dtype=int)
    return values[[f"{dim_name}_key", col]]

dim_rideable_type = make_dim(df, "rideable_type", "rideable_type")
dim_member_type   = make_dim(df, "member_casual", "member_type")
dim_time_of_day   = make_dim(df, "time_of_day", "time_of_day")

dim_rideable_type, dim_member_type, dim_time_of_day

(   rideable_type_key  rideable_type
 0                  1   classic_bike
 1                  2  electric_bike,
    member_type_key member_casual
 0                1        casual
 1                2        member,
    time_of_day_key time_of_day
 0                1   Afternoon
 1                2     Evening
 2                3     Morning
 3                4       Night)

In [22]:
fact = df.merge(dim_rideable_type, on="rideable_type", how="left")
fact = fact.merge(dim_member_type, on="member_casual", how="left")
fact = fact.merge(dim_time_of_day, on="time_of_day", how="left")

fact_rides = fact[[
    "ride_id",
    "date_key",
    "rideable_type_key",
    "member_type_key",
    "time_of_day_key",
    "start_station_name",
    "end_station_name",
    "start_hour",
    "duration_min",
    "source_file"
]].copy()

print("fact_rides shape:", fact_rides.shape)
fact_rides.head(5)

fact_rides shape: (610284, 10)


,ride_id,date_key,rideable_type_key,member_type_key,time_of_day_key,start_station_name,end_station_name,start_hour,duration_min,source_file
0,1,20250603,1,1,2,Montgomery St,Newport PATH,19,19.583050,JC-202506-citibike-tripdata.csv
1,2,20250608,2,1,2,Dixon Mills,Bergen Ave,20,10.129750,JC-202506-citibike-tripdata.csv
2,3,20250625,1,2,4,Bergen Ave & Sip Ave,Bergen Ave,22,6.284517,JC-202506-citibike-tripdata.csv
3,4,20250605,2,2,3,Hoboken Terminal - Hudson St & Hudson Pl,Newport PATH,5,5.377150,JC-202506-citibike-tripdata.csv
4,5,20250619,2,2,4,Hoboken Terminal - Hudson St & Hudson Pl,Bergen Ave,0,15.311150,JC-202506-citibike-tripdata.csv


In [23]:
fact_rides.to_csv(os.path.join(OUT_DIR, "fact_rides.csv"), index=False)
dim_date.to_csv(os.path.join(OUT_DIR, "dim_date.csv"), index=False)
dim_rideable_type.to_csv(os.path.join(OUT_DIR, "dim_rideable_type.csv"), index=False)
dim_member_type.to_csv(os.path.join(OUT_DIR, "dim_member_type.csv"), index=False)
dim_time_of_day.to_csv(os.path.join(OUT_DIR, "dim_time_of_day.csv"), index=False)

print("Saved to:", OUT_DIR)
print(sorted([os.path.basename(x) for x in glob.glob(os.path.join(OUT_DIR, '*.csv'))]))

Saved to: /content/outputs
['dim_date.csv', 'dim_member_type.csv', 'dim_rideable_type.csv', 'dim_time_of_day.csv', 'fact_rides.csv']
